In [1]:
from collections import defaultdict
from collections import Counter
import cv2
import numpy as np
from ultralytics import trackers
from ultralytics import YOLO
from ultralytics import RTDETR
import torch
from tracking_script import *

In [2]:
model = YOLO("/home/macula/SMATousi/Downloads/yolov8l-increase-augment-all.pt")
# model = RTDETR("/home/macula/SMATousi/Downloads/rtdetr-x-increase-augment-all.pt")

In [3]:
video_path = "/home/macula/SMATousi/Downloads/009.mp4"

In [4]:
trackers.basetrack.BaseTrack.reset_id()
# results = model.track(source=video_path, conf=0.1, iou=0.5, show=True, verbose=False, save_frames=True, save=True, project='./')
results = model.track(source=video_path, tracker='./bot_sort.yaml', conf=0.1, iou=0.1, show=False, save_conf = True, save_txt=True, verbose=False, save_frames=True, save=True, project='./tracking_results/results009')


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to tracking_results/results009/track
186 labels saved to tracking_results/results009/track/labels


In [5]:
# import pickle 

# with open("./results027-yolo-tracker/track11/results", "wb") as fp:
#     pickle.dump(results, fp)

In [6]:
iou_conditions_met = build_iou_array(results, num_frames = len(results))

In [7]:
iou_conditions_met[7,:10,:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [8]:
all_groups = identify_group(iou_conditions_met)

In [9]:
class_id_frame_map_array = class_id_frame_map(results, num_frames=len(results))

In [17]:
def modified_valid_group_identification(all_groups, appearance_threshold=0.2):
    all_group_list = []
    
    for frame in all_groups:
        # print(frame_group)
        for s in all_groups[frame]:
            # print(s)
            all_group_list.append(frozenset(s))
    
    frequencies = Counter(all_group_list)
    union_of_all_sets = set().union(*all_group_list)
    
    intersections_with_frequencies = {element: {} for element in union_of_all_sets}
    
    for element in union_of_all_sets:
        for group in all_group_list:
            intersection = group.intersection({element})
            if intersection:
                # If there is an intersection, get the frequency of this group
                group_as_set = set(group)  # Convert frozenset back to set for reporting
                frequency = frequencies[group]
                intersections_with_frequencies[element][str(group_as_set)] = frequency
    
    max_frequency_groups = {}

    for element, groups in intersections_with_frequencies.items():
        # Initialize variables to keep track of the max group and its frequency
        max_group = None
        max_frequency = -1
        max_group_size = 0  # Keep track of the size of the max group

        # Also track the most frequent group that has more than one member
        most_frequent_multi_member_group = None
        most_frequent_multi_member_group_frequency = -1

        total_appearance = 0
        
        for group_str, frequency in groups.items():

            total_appearance = total_appearance + frequency

        for group_str, frequency in groups.items():
            group_set = eval(group_str)  # Convert the string representation back to a set
            group_size = len(group_set)

            # Check if this group has the highest frequency seen so far
            if frequency > max_frequency:
                max_frequency = frequency
                max_group = group_str
                max_group_size = group_size

            # Independently check if this is the most frequent multi-member group
            if group_size == 2 and frequency > most_frequent_multi_member_group_frequency:
                most_frequent_multi_member_group_frequency = frequency
                most_frequent_multi_member_group = group_str

            if group_size > 2 and frequency/total_appearance > appearance_threshold:
                    most_frequent_multi_member_group_frequency = frequency
                    most_frequent_multi_member_group = group_str

        # Determine which group to return for this element
        if max_group_size > 1:
            max_frequency_groups[element] = max_group
        else:
            if most_frequent_multi_member_group is not None:
                max_frequency_groups[element] = most_frequent_multi_member_group
            else:
                max_frequency_groups[element] = max_group
    
    

    return max_frequency_groups, intersections_with_frequencies

In [18]:
max_frequency_groups, intersections_with_frequencies = modified_valid_group_identification(all_groups)

In [19]:
max_frequency_groups

{1: '{1, 3}',
 2: '{2, 5}',
 3: '{1, 3}',
 4: '{4, 7}',
 5: '{2, 5}',
 6: '{4, 6}',
 7: '{4, 7}',
 9: '{9, 4, 7}',
 13: '{13, 22, 31}',
 21: '{4, 21, 7}',
 22: '{13, 22, 31}',
 31: '{13, 22, 31}',
 33: '{33, 4, 21, 7}',
 37: '{41, 37}',
 41: '{41, 37}',
 46: '{48, 46}',
 48: '{48, 46}',
 55: '{56, 55}',
 56: '{56, 55}',
 59: '{59, 60, 61}',
 60: '{59, 60, 61}',
 61: '{59, 60, 61}'}

In [58]:
missing_members = find_missing_pair_member(max_frequency_groups, intersections_with_frequencies)

In [59]:
frame_ids_with_valid_group_for_missing = find_frame_and_valid_group_for_missing_pairs(all_groups, missing_members, max_frequency_groups)

In [228]:
def find_closest_frames_for_missing_id(all_groups, 
                                        max_frequency_groups,
                                        frame_ids_with_valid_group_for_missing):
    valid_groups = max_frequency_groups
    # Assuming 'frame_dict' is your initial dictionary with frame IDs as keys and lists of groups as values,
    # and 'valid_groups' is defined as shown previously.

    # Convert 'frame_dict' to have frozensets for easier comparison
    frame_dict_frozensets = {frame_id: [frozenset(group) for group in groups] for frame_id, groups in all_groups.items()}

    valid_group_to_frames = {str(group): [] for group in valid_groups.values()}
    for frame_id, groups in all_groups.items():
        for group in groups:
            group_str = str(group)
            if group_str in valid_group_to_frames:
                valid_group_to_frames[group_str].append(frame_id)

    # Convert lists to sorted sets for efficient lookup
    for group_str in valid_group_to_frames.keys():
        valid_group_to_frames[group_str] = sorted(set(valid_group_to_frames[group_str]))
    
    print(valid_group_to_frames)
    # Now, for each missing element, find the two closest frames with the valid group.
    closest_frames_for_missing = {}
    for missing_str, frame_infos in frame_ids_with_valid_group_for_missing.items():
        for info in frame_infos:
            frame_id = info['frame_id']
            valid_group_str = str(info['valid_group'])

            # Get all frames where the valid group appears
            valid_frames = valid_group_to_frames[valid_group_str]

            # Find the two closest frames (before or after) the current frame where the missing element is detected
            diffs = [(abs(frame_id - vf), vf) for vf in valid_frames]
            closest_two = sorted(diffs, key=lambda x: x[0])[:2]
            closest_frame_ids = [frame_id for _, frame_id in closest_two]

            # Store the results
            if missing_str not in closest_frames_for_missing:
                closest_frames_for_missing[missing_str] = []
            closest_frames_for_missing[missing_str].append({
                "missing_frame": frame_id,
                "closest_frames": closest_frame_ids
            })

    closest_frames_for_missing_with_completing_elements = {}

    for missing_str, frame_infos in frame_ids_with_valid_group_for_missing.items():
        missing_elements = eval(missing_str)  # Convert string back to a set
        for info in frame_infos:
            frame_id = info['frame_id']
            valid_group = eval(info['valid_group'])
            valid_group_str = str(valid_group)

            # Get all frames where the valid group appears
            valid_frames = valid_group_to_frames[valid_group_str]

            # Find the two closest frames (before or after) the current frame where the missing element is detected
            diffs = [(abs(frame_id - vf), vf) for vf in valid_frames]
            closest_two = sorted(diffs, key=lambda x: x[0])[:2]
            closest_frame_ids = [frame_id for _, frame_id in closest_two]

            # Find the elements that complete the missing element for valid groups
            completing_elements = valid_group - missing_elements

            # Store the results, including the completing elements
            if missing_str not in closest_frames_for_missing_with_completing_elements:
                closest_frames_for_missing_with_completing_elements[missing_str] = []
            closest_frames_for_missing_with_completing_elements[missing_str].append({
                "missing_frame": frame_id,
                "closest_frames": closest_frame_ids,
                "completing_elements": completing_elements
            })

    

    return closest_frames_for_missing_with_completing_elements

In [282]:
def find_closest_frames_for_missing_id(all_groups, max_frequency_groups, frame_ids_with_valid_group_for_missing):
    # Helper function to convert a group to a sorted tuple key
    def to_sorted_tuple_key(group):
        return tuple(sorted(group))
    valid_groups = max_frequency_groups
    # Convert all_groups to use sorted tuple keys
    frame_dict_tuples = {frame_id: [to_sorted_tuple_key(group) for group in groups] for frame_id, groups in all_groups.items()}
    
    # Initialize valid_group_to_frames using sorted tuple keys
    valid_group_to_frames = {}
    for frame_id, groups in frame_dict_tuples.items():
        for group in groups:
            if group in valid_group_to_frames:
                valid_group_to_frames[group].add(frame_id)  # Use a set to avoid duplicate frame IDs
            else:
                valid_group_to_frames[group] = {frame_id}

    # Now, process frame_ids_with_valid_group_for_missing, assuming it's been adjusted to use tuple keys as well.
    # Here we directly use the tuples, so make sure they're sorted and consistent.
    
    closest_frames_for_missing_with_completing_elements = {}
    for missing_str, frame_infos in frame_ids_with_valid_group_for_missing.items():
        missing_elements = eval(missing_str)  # Ensure this converts to a tuple if coming from a string
        for info in frame_infos:
            frame_id = info['frame_id']
            valid_group = to_sorted_tuple_key(eval(info['valid_group']))  # Convert to sorted tuple

            # Get all frames where the valid group appears
            valid_frames = sorted(valid_group_to_frames.get(valid_group, []))

            # Compute distances to find the closest frames
            diffs = [(abs(frame_id - vf), vf) for vf in valid_frames if vf != frame_id]
            closest_two = sorted(diffs, key=lambda x: x[0])[:2]
            closest_frame_ids = [vf for _, vf in closest_two]

            # Find the elements that complete the missing element for valid groups
            completing_elements = set(valid_group) - set(eval(missing_str))

            if missing_str not in closest_frames_for_missing_with_completing_elements:
                closest_frames_for_missing_with_completing_elements[missing_str] = []
            closest_frames_for_missing_with_completing_elements[missing_str].append({
                "missing_frame": frame_id,
                "closest_frames": closest_frame_ids,
                "completing_elements": completing_elements
            })

    return closest_frames_for_missing_with_completing_elements

# Note: Ensure that all input data is correctly formatted to work with this function.
# Specifically, 'all_groups' and 'frame_ids_with_valid_group_for_missing' need to be adapted to use sorted tuple keys.


In [60]:
closest_frames_for_missing_with_completing_elements = find_closest_frames_for_missing_id(all_groups, max_frequency_groups,frame_ids_with_valid_group_for_missing)

In [61]:
class_id_frame_corrections = correct_class_id(class_id_frame_map_array, threshold=0.9)

In [62]:
closest_frames_for_missing_with_completing_elements

{'{1}': [{'missing_frame': 28,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 29,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 30,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 31,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 32,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 33,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 34,
   'closest_frames': [27, 26],
   'completing_elements': {3}},
  {'missing_frame': 35,
   'closest_frames': [27, 26],
   'completing_elements': {3}}],
 '{7}': [{'missing_frame': 2,
   'closest_frames': [3, 7],
   'completing_elements': {4}}],
 '{4}': [{'missing_frame': 41,
   'closest_frames': [40, 34],
   'completing_elements': {7}},
  {'missing_frame': 42,
   'closest_frames': [40, 34],
   'completing_elements': {7}},
  {'missing_frame': 43,

In [167]:
def correct_post_1(text_files_path, video_id, class_id_frame_corrections):

    for i in range(class_id_frame_corrections.shape[0]):
        working_list = class_id_frame_corrections[i]
        if working_list[1] != working_list[2]:
            print("correction found!")
            frame_id = working_list[3]
            track_id = working_list[0]
            old_class = working_list[1]
            new_class = working_list[2]
            
            text_file_path = f"{text_files_path}/{video_id}_{frame_id+1}.txt"
            txt_class_ids, txt_boxes, txt_conf_scores, txt_track_ids = parse_text_file_corrected(text_file_path)
            line_position = np.where(txt_track_ids == track_id)[0][0]
            change_number_in_file(text_file_path, line_position + 1,new_class, number_position=1)
            print("Frame = ", frame_id)

In [285]:
for i in range(class_id_frame_corrections.shape[0]):
    working_list = class_id_frame_corrections[i]
    if working_list[1] != working_list[2]:
        print("correction found!")
        frame_id = working_list[3]
        track_id = working_list[0]
        old_class = working_list[1]
        new_class = working_list[2]

        # text_file_path = f"./track19/labels_post1/045_training_{frame_id+1}.txt"
        # txt_class_ids, txt_boxes, txt_conf_scores, txt_track_ids = parse_text_file_corrected(text_file_path)
        # line_position = np.where(txt_track_ids == track_id)[0][0]
        # change_number_in_file(text_file_path, line_position + 1,new_class, number_position=1)
        # print("Frame = ", frame_id)
    
    # break

correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!
correction found!


In [286]:
text_files_path = "./results009-final/track2/labels_pp2/"
correct_post_1(text_files_path, "009", class_id_frame_corrections)

correction found!
File updated successfully.
Frame =  20
correction found!
File updated successfully.
Frame =  21
correction found!
File updated successfully.
Frame =  70
correction found!
File updated successfully.
Frame =  18
correction found!
File updated successfully.
Frame =  19
correction found!
File updated successfully.
Frame =  20
correction found!
File updated successfully.
Frame =  21
correction found!
File updated successfully.
Frame =  22
correction found!
File updated successfully.
Frame =  123
correction found!
File updated successfully.
Frame =  126
correction found!
File updated successfully.
Frame =  81
correction found!
File updated successfully.
Frame =  53
correction found!
File updated successfully.
Frame =  139
correction found!
File updated successfully.
Frame =  154
correction found!
File updated successfully.
Frame =  177
correction found!
File updated successfully.
Frame =  178
correction found!
File updated successfully.
Frame =  181
correction found!
File u

In [63]:
text_files_path = "./results009-final/track2/labels_pp2/"
line_insert_post_2(results, text_files_path, "009", closest_frames_for_missing_with_completing_elements)

Corerection_list =  {'missing_frame': 28, 'closest_frames': [27, 26], 'completing_elements': {3}}
missing_frame =  28
successful!
[27, 26, 28]
Frame =  28 Id =  {1} intep =  [837.0420435844022, 217.37706570016408, 217.41573033707866, 344.175]
New line inserted successfully.
Corerection_list =  {'missing_frame': 29, 'closest_frames': [27, 26], 'completing_elements': {3}}
missing_frame =  29
successful!
[27, 26, 29]
Frame =  29 Id =  {1} intep =  [821.8318524826441, 195.49171563711198, 214.8876404494382, 341.23333333333335]
New line inserted successfully.
Corerection_list =  {'missing_frame': 30, 'closest_frames': [27, 26], 'completing_elements': {3}}
missing_frame =  30
successful!
[27, 26, 30]
Frame =  30 Id =  {1} intep =  [808.0229349200772, 180.84203421222838, 213.62359550561797, 335.35]
New line inserted successfully.
Corerection_list =  {'missing_frame': 31, 'closest_frames': [27, 26], 'completing_elements': {3}}
missing_frame =  31
successful!
[27, 26, 31]
Frame =  31 Id =  {1} i

In [263]:
closest_frames_for_missing_with_completing_elements

{'{21}': [{'missing_frame': 15,
   'closest_frames': [14, 13],
   'completing_elements': {6}},
  {'missing_frame': 16,
   'closest_frames': [14, 13],
   'completing_elements': {6}}],
 '{8}': [{'missing_frame': 35,
   'closest_frames': [34, 33],
   'completing_elements': {7}}],
 '{26, 5}': [{'missing_frame': 21,
   'closest_frames': [15, 14],
   'completing_elements': {9}},
  {'missing_frame': 21,
   'closest_frames': [15, 14],
   'completing_elements': {9}}],
 '{22}': [{'missing_frame': 23,
   'closest_frames': [22, 21],
   'completing_elements': {11}},
  {'missing_frame': 24,
   'closest_frames': [22, 21],
   'completing_elements': {11}}],
 '{4}': [{'missing_frame': 27,
   'closest_frames': [26, 25],
   'completing_elements': {12}},
  {'missing_frame': 28,
   'closest_frames': [26, 25],
   'completing_elements': {12}}],
 '{13}': [{'missing_frame': 10,
   'closest_frames': [9, 8],
   'completing_elements': {14}},
  {'missing_frame': 11, 'closest_frames': [9, 8], 'completing_elements': 

In [265]:
def line_insert_post_2(results, text_files_path, video_id, closest_frames_for_missing_with_completing_elements):
    
    for element in closest_frames_for_missing_with_completing_elements:
    
        red_boxes = []
        blue_boxes = []
        # (num,) = eval(element)
        # print(element)
        
        working_list = closest_frames_for_missing_with_completing_elements[element]
        for correction_list in working_list:
            print("Corerection_list = ", correction_list)
        
            try:
                missing = correction_list['missing_frame']
                closest_frames = correction_list['closest_frames']
                completing_elements = int(list(correction_list['completing_elements'])[0])
                print("missing_frame = ", missing)
                red_frames = closest_frames.copy()
                red_frames.append(missing)
            
                blue_frames = closest_frames.copy()
            
                red_cls, red_boxes = interpolation_known_points_extractor(results, completing_elements, red_frames)
                blue_cls, blue_boxes = interpolation_known_points_extractor(results, int(list(eval(element))[0]), blue_frames)
                # print(working_list)
            
    
    
                interpolated_array = triangular_interpolation(red_boxes, blue_boxes)
                print("successful!")
                print(red_frames) 
                print("Frame = ", missing, "Id = ", element, "intep = ", interpolated_array)
    
                text_file_path = f"{text_files_path}/{video_id}_{missing+1}.txt"
                missing_class = track_id_to_class_id(results,int(list(eval(element))[0]), nym_frames=200)
    
                insert_box = [interpolated_array[0]/1920, interpolated_array[1]/1080, interpolated_array[2]/1920, interpolated_array[3]/1080]
                insert_line_into_file(text_file_path, missing_class, insert_box, 1.0, int(list(eval(element))[0]), line_number=None) 
                
                # image = cv2.imread(f'/home/macula/SMATousi/ai_city/helmet/Deep-SORT-PyTorch/track18/045_training_frames//{missing + 1}.jpg')  
                # box = interpolated_array
    
                
                # image_with_box = draw_box(image.copy(), box, title=f"{int(list(eval(element))[0]), missing_class}")
                # image_with_box = cv2.resize(image_with_box, (960, 540))
                # # Display the image with the box
                # cv2.imshow('Image with Box', image_with_box)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
            except Exception as error:
                print("Error occured!: ", error)
                continue


In [98]:

def change_number_in_file(file_path, line_number,new_value, number_position=1):
    """
    Changes a specific number in a text file.

    :param file_path: The path to the text file.
    :param line_number: The line number to change (1-indexed).
    :param number_position: The position of the number in the line to change (1-indexed).
    :param new_value: The new value for the number.
    """
    try:
        # Read all lines from the file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Check if the specified line number is within the range of the file's lines
        if line_number > len(lines) or line_number < 1:
            print("Error: The specified line number is out of the file's range.")
            return
        
        # Select the specific line
        line = lines[line_number - 1]
        parts = line.strip().split()
        
        # Check if the specified number position is within the range of the line's parts
        if number_position > len(parts) or number_position < 1:
            print("Error: The specified number position is out of the line's range.")
            return
        
        # Change the specified number
        parts[number_position - 1] = str(new_value)
        # Reconstruct the line and update it in the lines list
        lines[line_number - 1] = ' '.join(parts) + '\n'
        
        # Write the modified lines back to the file
        with open(file_path, 'w') as file:
            file.writelines(lines)
        
        print("File updated successfully.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

In [90]:
frame_id

180

In [30]:
np.where(working_tracking_results[frame_id].boxes.id.cpu().numpy() == track_id)

(array([4]),)

In [48]:
working_tracking_results[frame_id].boxes.id

tensor([1., 2., 3., 4., 5.])

In [43]:
print(correction_index[0][0])

4


In [36]:

for element in closest_frames_for_missing_with_completing_elements:

    red_boxes = []
    blue_boxes = []
    # (num,) = eval(element)
    # print(element)
    
    working_list = closest_frames_for_missing_with_completing_elements[element]
    for correction_list in working_list:
        print("Corerection_list = ", correction_list)
    
        try:
            missing = correction_list['missing_frame']
            closest_frames = correction_list['closest_frames']
            completing_elements = int(list(correction_list['completing_elements'])[0])
            print("missing_frame = ", missing)
            red_frames = closest_frames.copy()
            red_frames.append(missing)
        
            blue_frames = closest_frames.copy()
        
            red_cls, red_boxes = interpolation_known_points_extractor(results, completing_elements, red_frames)
            blue_cls, blue_boxes = interpolation_known_points_extractor(results, int(list(eval(element))[0]), blue_frames)
            # print(working_list)
        


            interpolated_array = triangular_interpolation(red_boxes, blue_boxes)
            print("successful!")
            print(red_frames) 
            print("Frame = ", missing, "Id = ", element, "intep = ", interpolated_array)

            # text_file_path = f"{text_files_path}/{video_id}_{missing+1}.txt"
            missing_class = track_id_to_class_id(results,int(list(eval(element))[0]), nym_frames=len(results))

            # insert_box = [interpolated_array[0]/1920, interpolated_array[1]/1080, interpolated_array[2]/1920, interpolated_array[3]/1080]
            # insert_line_into_file(text_file_path, missing_class, insert_box, 1.0, int(list(eval(element))[0]), line_number=None) 
            
            image = cv2.imread(f'./results009-debug/track/009_frames/{missing+1}.jpg')  
            box = interpolated_array

            
            image_with_box = draw_box(image.copy(), box, title=f"{int(list(eval(element))[0]), missing_class, missing}")
            image_with_box = cv2.resize(image_with_box, (960, 540))
            # Display the image with the box
            cv2.imshow('Image with Box', image_with_box)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
        except Exception as e:
            print("Error occured!:", e)
            continue
    # break

Corerection_list =  {'missing_frame': 28, 'closest_frames': [27, 26], 'completing_elements': {4}}
missing_frame =  28
successful!
[27, 26, 28]
Frame =  28 Id =  {1} intep =  [839.6039173779762, 211.35940887258784, 249.41176470588235, 344.21487603305786]
Corerection_list =  {'missing_frame': 29, 'closest_frames': [27, 26], 'completing_elements': {4}}
missing_frame =  29
successful!
[27, 26, 29]
Frame =  29 Id =  {1} intep =  [822.5207874698335, 191.4718784687921, 245.88235294117646, 345.6611570247934]
Corerection_list =  {'missing_frame': 30, 'closest_frames': [27, 26], 'completing_elements': {4}}
missing_frame =  30
successful!
[27, 26, 30]
Frame =  30 Id =  {1} intep =  [808.6821563368737, 177.54767319692286, 240.0, 339.87603305785126]
Corerection_list =  {'missing_frame': 31, 'closest_frames': [27, 26], 'completing_elements': {4}}
missing_frame =  31
successful!
[27, 26, 31]
Frame =  31 Id =  {1} intep =  [792.65281625565, 159.35207434705242, 236.47058823529412, 336.9834710743802]
Co

KeyboardInterrupt: 

In [37]:
missing_class

1.0

In [46]:
def triangular_interpolation_debug(red_boxes, blue_boxes):

    red_box_1 = red_boxes[0]
    red_box_2 = red_boxes[1]
    red_box_3 = red_boxes[2]

    print("red boxes = ",red_box_1,red_box_2,red_box_3)
    
    
    blue_box_1 = blue_boxes[0]
    blue_box_2 = blue_boxes[1]

    print("blue boxes = ",blue_box_1,blue_box_2)

    x_red_1 = red_box_1[0]
    y_red_1 = red_box_1[1]

    print("red x1y1 = ",x_red_1,y_red_1)
    
    x_red_2 = red_box_2[0]
    y_red_2 = red_box_2[1]

    print("red x2y2 = ",x_red_2,y_red_2)
    
    x_red_3 = red_box_3[0]
    y_red_3 = red_box_3[1]

    print("red x3y3 = ",x_red_3,y_red_3)
    
    alpha1 = np.arctan2((y_red_2 - y_red_1),(x_red_2 - x_red_1))
    r1_mag = np.sqrt((y_red_2 - y_red_1)**2 + (x_red_2 - x_red_1)**2)

    print("alpha1, r1 = ", alpha1, r1_mag)
    
    alpha2 = np.arctan2((y_red_3 - y_red_2),(x_red_3 - x_red_2))
    r2_mag = np.sqrt((y_red_3 - y_red_2)**2 + (x_red_3 - x_red_2)**2)

    print("alpha2, r2 = ", alpha2, r2_mag)
    
    alpha_g = alpha2 - alpha1

    print("alpha_g = ", alpha_g)
    
    x_blue_1 = blue_box_1[0]
    y_blue_1 = blue_box_1[1]

    print("blue x1y1 = ",x_blue_1,y_blue_1)
    
    x_blue_2 = blue_box_2[0]
    y_blue_2 = blue_box_2[1]

    print("blue x2y2 = ",x_blue_2,y_blue_2)
    
    beta_1 = np.arctan2((y_blue_2 - y_blue_1),(x_blue_2 - x_blue_1))
    b1_mag = np.sqrt((y_blue_2 - y_blue_1)**2 + (x_blue_2 - x_blue_1)**2)

    print("beta1, b1 = ", beta_1, b1_mag)
    
    beta2 = beta_1 + alpha_g
    b2_mag = (r2_mag * b1_mag)/r1_mag

    print("beta2, b2 = ", beta2, b2_mag)
    
    x_blue_3 = x_blue_2 + b2_mag * np.cos(beta2)
    y_blue_3 = y_blue_2 + b2_mag * np.sin(beta2)

    print("cos = ", np.cos(beta2))
    print("sin = ", np.sin(beta2))
    
    print("blue x3y3 = ", x_blue_3, y_blue_3)

    w_red_2 = red_box_2[2]
    h_red_2 = red_box_2[3]
    
    w_red_3 = red_box_3[2]
    h_red_3 = red_box_3[3]
    
    w_blue_2 = blue_box_2[2]
    h_blue_2 = blue_box_2[3]
    
    w_blue_3 = (w_red_3 * w_blue_2)/w_red_2
    h_blue_3 = (h_red_3 * h_blue_2)/h_red_2

    print("w,h inter = ", w_blue_3, h_blue_3)
    
    return [x_blue_3, y_blue_3, w_blue_3, h_blue_3]

In [47]:
red_boxes

[array([ 66, 328,  88, 241], dtype=int32),
 array([ 65, 343,  86, 246], dtype=int32),
 array([ 86, 240, 113, 215], dtype=int32)]

In [48]:
blue_boxes

[array([ 81, 235,  96, 308], dtype=int32),
 array([ 76, 225, 100, 347], dtype=int32)]

In [49]:
triangular_interpolation_debug(red_boxes, blue_boxes)

red boxes =  [ 66 328  88 241] [ 65 343  86 246] [ 86 240 113 215]
blue boxes =  [ 81 235  96 308] [ 76 225 100 347]
red x1y1 =  66 328
red x2y2 =  65 343
red x3y3 =  86 240
alpha1, r1 =  1.6373644905707205 15.033296378372908
alpha2, r2 =  -1.3696694409783572 105.11898020814318
alpha_g =  -3.0070339315490777
blue x1y1 =  81 235
blue x2y2 =  76 225
beta1, b1 =  -2.0344439357957027 11.180339887498949
beta2, b2 =  -5.04147786734478 78.17752658991466
cos =  0.3231809425002084
sin =  0.9463371906485959
blue x3y3 =  101.26548672566372 298.98230088495575
w,h inter =  131.3953488372093 303.2723577235772


[101.26548672566372, 298.98230088495575, 131.3953488372093, 303.2723577235772]

In [134]:
def insert_line_into_file(file_path, class_id, box_values, conf_score, track_id, line_number=None):
    """
    Inserts a new line into the text file with the specified class ID, box values, and confidence score.

    :param file_path: The path to the text file.
    :param class_id: The class ID as an integer.
    :param box_values: The box values as a list of four numbers (coordinates).
    :param conf_score: The confidence score as a float.
    :param line_number: Optional; the line number at which to insert the new line (1-indexed). If not specified or
                        if the line number is greater than the file length, the new line will be appended at the end.
    """
    new_line = f"{class_id} {' '.join(map(str, box_values))} {conf_score} {track_id}\n"
    
    try:
        # Read all lines from the file
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # Determine where to insert the new line
        if line_number is not None and 1 <= line_number <= len(lines) + 1:
            lines.insert(line_number - 1, new_line)
        else:
            # Append at the end if no valid line number is provided or it's beyond the file length
            lines.append(new_line)
        
        # Write the lines back to the file
        with open(file_path, 'w') as file:
            file.writelines(lines)
        
        print("New line inserted successfully.")
        
    except Exception as e:
        print(f"An error occurred: {e}")


In [135]:
file_path = "./test_line.txt"
insert_line_into_file(file_path, 1, interpolated_array, 1, 2, line_number=None) 

New line inserted successfully.


In [129]:
interpolated_array

[602.4694975272112, 154.86471856527027, 64.51162790697674, 100.625]

In [57]:
results[14].boxes
results[14].boxes.cls = torch.tensor([0., 0.])

AttributeError: property 'cls' of 'Boxes' object has no setter

In [242]:
def triangular_interpolation(red_boxes, blue_boxes):

    red_box_1 = red_boxes[0]
    red_box_2 = red_boxes[1]
    red_box_3 = red_boxes[2]
    
    blue_box_1 = blue_boxes[0]
    blue_box_2 = blue_boxes[1]

    x_red_1 = red_box_1[0]
    y_red_1 = red_box_1[1]
    
    x_red_2 = red_box_2[0]
    y_red_2 = red_box_2[1]
    
    x_red_3 = red_box_3[0]
    y_red_3 = red_box_3[1]
    
    alpha1 = np.arctan2((y_red_2 - y_red_1),(x_red_2 - x_red_1 + 0.00001))
    r1_mag = np.sqrt((y_red_2 - y_red_1)**2 + (x_red_2 - x_red_1)**2)
    
    alpha2 = np.arctan2((y_red_3 - y_red_2),(x_red_3 - x_red_2 + 0.00001))
    r2_mag = np.sqrt((y_red_3 - y_red_2)**2 + (x_red_3 - x_red_2)**2)
    
    alpha_g = alpha2 - alpha1

    
    x_blue_1 = blue_box_1[0]
    y_blue_1 = blue_box_1[1]
    
    x_blue_2 = blue_box_2[0]
    y_blue_2 = blue_box_2[1]
    
    beta_1 = np.arctan2((y_blue_2 - y_blue_1),(x_blue_2 - x_blue_1 + 0.00001))
    b1_mag = np.sqrt((y_blue_2 - y_blue_1)**2 + (x_blue_2 - x_blue_1)**2)
    
    beta2 = beta_1 + alpha_g
    b2_mag = (r2_mag * b1_mag)/r1_mag
    
    x_blue_3 = x_blue_2 + b2_mag * np.cos(beta2)
    y_blue_3 = y_blue_2 + b2_mag * np.sin(beta2)

    w_red_2 = red_box_2[2]
    h_red_2 = red_box_2[3]
    
    w_red_3 = red_box_3[2]
    h_red_3 = red_box_3[3]
    
    w_blue_2 = blue_box_2[2]
    h_blue_2 = blue_box_2[3]
    
    w_blue_3 = (w_red_3 * w_blue_2)/w_red_2
    h_blue_3 = (h_red_3 * h_blue_2)/h_red_2
    
    return [x_blue_3, y_blue_3, w_blue_3, h_blue_3]


In [46]:
max_frequency_groups

{1: '{1, 3}',
 2: '{2, 5}',
 3: '{1, 3}',
 4: '{4, 5}',
 5: '{4, 5}',
 6: '{8, 6}',
 8: '{8, 6}',
 9: '{8, 9}',
 12: '{12, 15}',
 14: '{24, 14}',
 15: '{12, 15}',
 18: '{18, 19}',
 19: '{18, 19}',
 21: '{21, 23}',
 23: '{30, 23}',
 24: '{24, 14}',
 25: '{32, 25}',
 26: '{26, 27}',
 27: '{26, 27}',
 30: '{30, 23}',
 31: '{21, 31}',
 32: '{32, 25}',
 34: '{34, 30, 23}',
 35: '{35, 36}',
 36: '{35, 36}',
 39: '{41, 39}',
 40: '{40}',
 41: '{41, 39}'}

In [98]:
intersections_with_frequencies

{1: {'{1}': 27},
 2: {'{2}': 5, '{2, 5}': 5},
 3: {'{3}': 27},
 4: {'{4}': 27},
 5: {'{2, 5}': 5, '{5}': 17},
 6: {'{6}': 33},
 7: {'{7}': 33},
 10: {'{10}': 20},
 11: {'{11}': 13},
 12: {'{12}': 20},
 14: {'{14}': 13},
 15: {'{15}': 21},
 16: {'{16}': 21},
 17: {'{17}': 21},
 18: {'{18, 19}': 2, '{18}': 20},
 19: {'{19}': 19, '{18, 19}': 2},
 20: {'{20}': 6, '{25, 20}': 1},
 24: {'{24}': 6},
 25: {'{25}': 19, '{25, 20}': 1},
 27: {'{27}': 12},
 29: {'{29}': 12},
 31: {'{31}': 38},
 33: {'{33}': 38},
 35: {'{35}': 23},
 36: {'{36}': 23}}

In [17]:
all_groups

{0: [{1, 3}, {2, 4}],
 1: [{1, 3}, {2, 4}],
 2: [{1, 3}, {2, 4}],
 3: [{1, 3}, {2}],
 4: [{1, 3}, {2}],
 5: [{1, 3}, {2}],
 6: [{1, 3}, {2, 4}],
 7: [{1, 3}, {2, 4}],
 8: [{1, 3}, {2, 4}, {2, 4, 5}, {4, 5}],
 9: [{1, 3}, {2, 4}, {2, 4, 5}, {4, 5}],
 10: [{1, 3}, {2, 4}, {2, 4, 5}, {4, 5}],
 11: [{1, 3}, {2, 4}, {2, 4, 5}, {4, 5}],
 12: [{1, 3}, {2, 4}, {2, 4, 5}, {4, 5}],
 13: [{1, 3}, {2}, {5}, {6}],
 14: [{1, 3}, {4, 5}, {6}],
 15: [{1, 3}, {4, 5}, {6}],
 16: [{4, 5}, {6, 7}],
 17: [{1, 3}, {4, 5}, {6, 7}],
 18: [{1, 3}, {4, 5}, {6, 7}],
 19: [{1, 3}, {4, 5}, {6, 7}],
 20: [{1, 3}, {4, 5}, {6, 7}],
 21: [{1, 3}, {4, 5}, {6, 7}],
 22: [{1, 3}, {4, 5}, {6, 7}],
 23: [{1, 3}, {4, 5}, {6, 7}],
 24: [{1, 3}, {4, 5}, {6, 7}],
 25: [{1, 3}, {4, 5}, {6, 7}],
 26: [{1, 3}, {4, 5}, {6, 7}],
 27: [{1, 3}, {4, 5}, {6, 7}],
 28: [{1}, {4, 5}, {6, 7}],
 29: [{4, 5}, {6, 7}],
 30: [{4, 5}, {6, 7}],
 31: [{6, 7}],
 32: [{6, 7}],
 33: [{6, 7}],
 34: [{6, 7}],
 35: [{6, 7}],
 36: [{6, 7}],
 37: [{6, 7

In [35]:
def draw_box(image, box, title):
    center = box[:2]
    # print(center)
    w = box[2]
    h = box[3]
    # Calculate top-left corner coordinates
    x = int(center[0] - w/2)
    y = int(center[1] - h/2)
    # print(x)
    # Draw box on the image
    cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
    cv2.putText(image, title, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return image

In [89]:
image = cv2.imread('/home/macula/SMATousi/ai_city/helmet/Deep-SORT-PyTorch/track8/045_frames/86.jpg')  # Replace 'image.jpg' with your image path

# Define center coordinates and box dimensions (width and height)
# box = [1854.865836791148, 900.9197786998617, 181.4142857142857, 342.29468599033817]  # Example center coordinates, adjust as needed
box = [520.0123393316195, 264.9809768637532, 248.92193308550185, 332.4518272425249]

# Draw box on the image
image_with_box = draw_box(image.copy(), box)
image_with_box = cv2.resize(image_with_box, (960, 540))
# Display the image with the box
cv2.imshow('Image with Box', image_with_box)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [69]:
results[114].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 2., 0., 0., 0., 1., 1.])
conf: tensor([0.4967, 0.4464, 0.5088, 0.8169, 0.8215, 0.7992, 0.8126])
data: tensor([[1.7531e+03, 4.3361e+02, 1.9200e+03, 8.1760e+02, 1.6000e+01, 4.9668e-01, 1.0000e+00],
        [9.2605e+02, 2.5709e+02, 1.2321e+03, 6.5964e+02, 1.7000e+01, 4.4636e-01, 2.0000e+00],
        [9.1987e+02, 4.2361e+02, 1.2410e+03, 6.6794e+02, 1.9000e+01, 5.0878e-01, 0.0000e+00],
        [4.4261e+02, 2.9925e+02, 7.4916e+02, 6.6582e+02, 2.0000e+01, 8.1695e-01, 0.0000e+00],
        [6.9146e+02, 1.3978e+02, 9.5110e+02, 4.5923e+02, 1.8000e+01, 8.2146e-01, 0.0000e+00],
        [4.5040e+02, 1.8717e+02, 7.4946e+02, 5.7118e+02, 2.4000e+01, 7.9917e-01, 1.0000e+00],
        [7.1249e+02, 3.0912e+01, 9.8285e+02, 4.0145e+02, 2.5000e+01, 8.1260e-01, 1.0000e+00]])
id: tensor([16., 17., 19., 20., 18., 24., 25.])
is_track: True
orig_shape: (1080, 1920)
shape: torch.Size([7, 7])
xywh: tensor([[1836.5383,  625.6036,  166.9232,  3

In [49]:
import pickle

In [138]:
def track_id_to_class_id(results, track_id, nym_frames=200):
    all_track_ids = []
    all_class_ids = []
    all_boxes = []

    for frame in range(nym_frames):
        working_results = results[frame]
        try:
            all_track_ids.extend(working_results.boxes.id.cpu().tolist())
            all_class_ids.extend(working_results.boxes.cls.cpu().tolist())
            all_boxes.extend(working_results.boxes.xywh.cpu().tolist())
        except:
            continue

    all_track_ids = np.array(all_track_ids)
    all_class_ids = np.array(all_class_ids)
    # all_boxes = np.array(all_boxes)

    des_index = np.where(all_track_ids == track_id)
    class_id = all_class_ids[des_index[0][0]]
    
    return class_id

In [63]:
track_id_to_class_id(results,23, nym_frames=200)

array([[     477.67,      335.88,      61.923,      133.92],
       [     486.95,      353.56,      65.182,      140.57],
       [     496.66,       369.3,      65.685,      140.58],
       ...,
       [     843.82,      558.25,      154.12,      244.65],
       [     848.92,      678.29,      138.46,      204.45],
       [     836.78,      582.74,      158.16,      251.47]])

In [84]:
def parse_text_file_corrected(file_path):
    """
    Correctly parses a text file to extract class IDs, box coordinates, and confidence scores.
    
    :param file_path: The path to the text file.
    :return: A tuple of lists containing class IDs, box coordinates, and confidence scores.
    """
    class_ids = []
    boxes = []
    conf_scores = []
    track_ids = []

    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            
            if not lines:
                print("The file is empty or could not be read.")
            
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 7:
                    print(f"Skipping line due to incorrect format: {line.strip()}")
                    continue  # Skip lines that don't have exactly 7 parts
                
                # Adjusting based on the new understanding of the format
                box = list(map(float, parts[:5]))  # First 5 parts are the box coordinates
                conf_score = float(parts[5])  # Second to last part is the confidence score
                class_id = int(parts[0])  # Last part is the class ID
                track_id = int(parts[6])

                class_ids.append(class_id)
                boxes.append(box)
                conf_scores.append(conf_score)
                track_ids.append(track_id)
    except Exception as e:
        print(f"An error occurred: {e}")

    return class_ids, boxes, conf_scores, track_ids

In [85]:
parse_text_file_corrected("./track19/labels/045_training_10.txt")

([0, 2],
 [[0.0, 0.282058, 0.394548, 0.0397874, 0.143083],
  [2.0, 0.278523, 0.331478, 0.0467579, 0.154689]],
 [0.787568, 0.742497],
 [1, 4])